# ONE Quick Start
This tutorial will get you started searching and loading IBL data using Open Neurophysiology
Environment (ONE).

First we need to install ONE. If you don't already have IBL libraries, the easiest way is to run
 `pip install ONE-api`.

Now we need to import the ONE library and open a connection to the IBL public data server. To do
so, we create an ONE object, and ask it to connect to the IBL public server.

<div class="alert alert-info">
Info.

IBL internal users may use their Alyx credentials to access all IBL data.
[Click here](https://int-brain-lab.github.io/ONE/one_installation.html#connecting-to-specific-database-relevant-for-ibl-users) for details.
</div>

In [ ]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)

Now we are going to search for an experiment to analyze. First let's find out what we can search by:

In [ ]:
print(one.search_terms())

Let's search for sessions recorded in September 2020, which contain a dataset 'probes.description',
meaning that electrophysiology was recorded. By adding the argument `details=True`, we get two
outputs - the experiment IDs uniquely identifying these sessions, and some information about the
experiments.

In [ ]:
eids, info = one.search(date_range=['2020-09-01', '2021-09-30'], dataset='probes.description', details=True)

from pprint import pprint
pprint(eids)
pprint(info)

So there were two experiments matching the criteria in the public database. Now let's load the
probe information for the first experiment. `one.load_dataset` returns a single named dataset

In [ ]:
eid = eids[0]  # Select the first experiment
probe_insertions = one.load_dataset(eid, 'probes.description')

print(f'N probes = {len(probe_insertions)}')
pprint(probe_insertions[0])

Now let's see all the datasets associated with the first of these experiments. The command `one.list_datasets`
returns the full path of all datasets, including the collection name and the extension. The
'alf' collection contains the preprocessed data we usually want to work with, and the data for
each probe are in labeled subdirectories:

In [ ]:
probe_label = probe_insertions[0]['label']
one.list_datasets(eid, collection=f'alf/{probe_label}')

Here we'll load the electrophysiology information. There are two ways to do this.
`one.load_dataset` or `one.load_object`, which returns all the datasets with the same name part,
as an object. Let's use the second method to load the spike times, depths, and amplitudes for
probe00:

In [ ]:
spikes = one.load_object(eids[0], 'spikes',
                         collection='alf/probe00', attribute=['times', 'depths', 'amps'])
spikes

We can import a module from brainbox that plots the raster across time
<div class="alert alert-info">
Note.

Requires [ibllib](https://github.com/int-brain-lab/ibllib#readme) to be installed.
</div>

In [ ]:
from brainbox.plot import driftmap
driftmap(spikes.times, spikes.depths, t_bin=0.1, d_bin=5);

And that's the end of the quick start tutorial! For more information on any of these commands you
can use the standard help function:

In [ ]:
help(one.list_datasets)

For detailed tutorials, guides and examples, [click here for the full ONE API documentation
Website](https://int-brain-lab.github.io/ONE/).